In [1]:
import sys
import os
from pathlib import Path

# Это нужно сделать ОДИН РАЗ в начале ноутбука
# Находим корень проекта (на два уровня выше текущей директории ноутбука)
# и добавляем папку src в пути для импорта
project_root = Path.cwd().parent 
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from local_paths import RAW_DATA_DIR, FINAL_DATA_DIR
import pandas as pd

df = pd.read_parquet(FINAL_DATA_DIR / 'combined.parquet')
df.head()

,symbol,date,open,high,low,close,volume,adj_close,close_mean_3d,close_std_3d,...,close_mean_30d,close_std_30d,close_mean_60d,close_std_60d,volume_mean_7d,log_return_1d,price_range_1d,company_age_days,up_streak,down_streak
0,A,1999-11-18,32.546494,35.765381,28.612303,31.473534,62546380,26.347027,31.473534,NaN,...,31.473534,NaN,31.473534,NaN,6.254638e+07,NaN,0.227273,0,0,0
1,A,1999-11-19,30.713518,30.758226,28.478184,28.880545,15234146,24.176392,30.177039,1.833520,...,30.177039,1.833520,30.177039,1.833520,3.889026e+07,-0.085979,0.078947,1,0,1
2,A,1999-11-22,29.551144,31.473534,28.657009,31.473534,6577870,26.347027,30.609204,1.497063,...,30.609204,1.497063,30.609204,1.497063,2.811947e+07,0.085979,0.089489,4,1,0
3,A,1999-11-23,30.400572,31.205294,28.612303,28.612303,5975611,23.951838,29.655460,1.580200,...,30.109979,1.578301,30.109979,1.578301,2.258350e+07,-0.095310,0.090625,5,0,1
4,A,1999-11-24,28.701717,29.998213,28.612303,29.372318,4843231,24.588066,29.819385,1.482080,...,29.962447,1.406096,29.962447,1.406096,1.903545e+07,0.026216,0.047184,6,1,0


In [6]:
# Начальный капитал (V0) во всех случаях = 1000.0

# 1. Положительный тренд: Финал ~2000 (x2)
portfolio_uptrend = [
    1000.0, 1021.45, 1054.3, 1032.1, 1085.6, 1120.45, 1095.3, 1140.2, 
    1185.1, 1210.4, 1195.8, 1245.3, 1280.9, 1315.4, 1290.15, 1345.6, 
    1390.25, 1425.8, 1395.1, 1455.4, 1510.3, 1565.2, 1530.45, 1590.8, 
    1655.1, 1710.4, 1785.25, 1850.6, 1915.15, 1960.3, 2005.1
]

# 2. Боковой тренд (Flat): Финал ~1000 (x1)
portfolio_flat = [
    1000.0, 985.12, 1012.4, 995.6, 978.2, 1025.15, 1034.8, 967.45, 
    992.1, 1015.3, 1045.6, 1021.1, 988.45, 965.2, 1012.9, 990.15, 
    1032.4, 975.6, 954.2, 1018.15, 1045.3, 988.7, 962.15, 1025.4, 
    1010.2, 982.15, 1035.6, 995.1, 1012.45, 985.3, 998.45
]

# 3. Отрицательный тренд: Финал ~500 (x0.5)
portfolio_downtrend = [
    1000.0, 972.1, 984.3, 945.15, 912.4, 885.6, 902.1, 865.3, 
    834.15, 856.2, 821.4, 795.8, 772.3, 798.15, 754.6, 732.1, 
    705.45, 682.3, 712.1, 695.4, 665.15, 632.4, 610.1, 585.3, 
    615.45, 584.2, 556.1, 534.8, 512.15, 525.4, 498.9
]


def calc_reward(day_before, day_now):
    return (day_now - day_before) / day_before

for portfolio in [portfolio_uptrend, portfolio_flat, portfolio_downtrend]:
    rewards = []
    for i in range(1, len(portfolio)):
        reward = calc_reward(portfolio[i-1], portfolio[i])
        rewards.append(reward)
        if i % 5 == 0:
            print(f"Награда: {portfolio[i-1]:.1f} -> {portfolio[i]:.1f} = {reward:.6f}")
    avg_reward = sum(rewards) / len(rewards)
    print(f'Average Reward: {avg_reward:.6f}')
    print(f"Sum Reward: {sum(rewards):.6f}\n")

Награда: 1085.6 -> 1120.5 = 0.032102
Награда: 1210.4 -> 1195.8 = -0.012062
Награда: 1290.2 -> 1345.6 = 0.042979
Награда: 1455.4 -> 1510.3 = 0.037722
Награда: 1655.1 -> 1710.4 = 0.033412
Награда: 1960.3 -> 2005.1 = 0.022854
Average Reward: 0.023722
Sum Reward: 0.711660

Награда: 978.2 -> 1025.2 = 0.047996
Награда: 1015.3 -> 1045.6 = 0.029843
Награда: 1012.9 -> 990.1 = -0.022460
Награда: 1018.1 -> 1045.3 = 0.026666
Награда: 1010.2 -> 982.1 = -0.027767
Награда: 985.3 -> 998.5 = 0.013346
Average Reward: 0.000621
Sum Reward: 0.018634

Награда: 912.4 -> 885.6 = -0.029373
Награда: 856.2 -> 821.4 = -0.040645
Награда: 754.6 -> 732.1 = -0.029817
Награда: 695.4 -> 665.1 = -0.043500
Награда: 615.5 -> 584.2 = -0.050776
Награда: 525.4 -> 498.9 = -0.050438
Average Reward: -0.022445
Sum Reward: -0.673353

